

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_NIHSS.ipynb)




# **Extract neurologic deficits related to Stroke Scale (NIHSS)**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

**Import license keys**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## 2. Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

In [ ]:
spark

## 3. Select the NER model and construct the pipeline

In [ ]:
MODEL_NAME = "ner_nihss"

**Create the pipeline**

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = nlp.SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = nlp.Tokenizer()\
    .setInputCols(['sentence']) \
    .setOutputCol('token')

word_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models') \
    .setInputCols(['sentence', 'token']) \
    .setOutputCol('embeddings')

clinical_ner = medical.NerModel.pretrained(MODEL_NAME, "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[document_assembler, 
                                sentence_detector,
                                tokenizer,
                                word_embeddings,
                                clinical_ner,
                                ner_converter])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_nihss download started this may take some time.
[OK!]


## 4. Create example inputs

In [ ]:
sample_text = ["""Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , one to two for gaze , two for face , eight for motor , one for limited ataxia , one to two for sensory , three for best language . On the neurologic examination the patient was intermittently"""]

## 5. Use the pipeline to create outputs

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text,StringType()).toDF('text')
result = nlp_pipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(30, truncate=False)

+----------------+---------------+
|chunk           |ner_label      |
+----------------+---------------+
|NIH stroke scale|NIHSS          |
|23 to 24        |Measurement    |
|one             |Measurement    |
|consciousness   |1a_LOC         |
|two             |Measurement    |
|month and year  |1b_LOCQuestions|
|two             |Measurement    |
|eye / grip      |1c_LOCCommands |
|one             |Measurement    |
|two             |Measurement    |
|gaze            |2_BestGaze     |
|two             |Measurement    |
|face            |4_FacialPalsy  |
|eight           |Measurement    |
|one             |Measurement    |
|limited ataxia  |7_LimbAtaxia   |
|one to two      |Measurement    |
|sensory         |8_Sensory      |
|three           |Measurement    |
|best language . |9_BestLanguage |
+----------------+---------------+



## 6. Visualize results

In [ ]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)